In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [60]:
ls /datalab

run.sh*  web/


In [15]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/workshop/keras-models/src/main/py')

In [16]:
import configparser
import os

import tensorflow as tf

from nlp.corpus.reader import DataProcessor
from nlp.segment.model import build_glove_bi_gru_crf_model, build_glove_bi_gru2_crf_model, MAX_LENGTH
from nlp.segment.seg import SegmentBase
from util import ObjectDict

In [19]:
build_model = build_glove_bi_gru2_crf_model

In [17]:
class CRFSegment(SegmentBase):
    def __init__(self, args, dataset, mode='seg'):
        self.model = build_model(args, dataset.word_to_id)
        self.model_path = args.model_path

        if mode == 'seg':
            self.model.load_weights(self.model_path)
        SegmentBase.__init__(self, self.model, dataset)

    def train(self, train_x, train_y):
        self.model.fit(
            train_x, train_y,
            batch_size=512,
            epochs=10,
            validation_split=0.1
        )
        self.model.save(self.model_path)

In [42]:
config = configparser.ConfigParser()
config.read(os.path.join('', "model-args-config.ini"))

data_path = '/content/gdrive/My Drive/data'
model_path = 'nlp/model/keras-models'
corpus_path = 'nlp/corpus/icwb2-data/training/msr_training.utf8'
glove_path = 'nlp/embidding/glove/tencent/Tencent_AILab_ChineseEmbedding_Single.txt'

dataset = DataProcessor()

In [53]:
def build_segment():
    mode = 'seg'
    mode = 'train'
    model_name = 'seg-glove-bi-gru2-crf-model-mask'

    args = ObjectDict()
    args.vocab_size = dataset.vocab_size
    args.num_labels = dataset.num_labels

    args.model_path = os.path.join(data_path, model_path, model_name + '.h5')
    args.glove_path = os.path.join(data_path, glove_path)

    crf_seg = CRFSegment(args, dataset, mode)
    return crf_seg

In [20]:
if __name__ == '__main__':

    crf_seg = build_segment()
    if mode == 'train':
        train_data_path = os.path.join(data_path, corpus_path)
        train_x, train_y = dataset.process_input_data(train_data_path, maxlen=MAX_LENGTH)
        crf_seg.train(train_x, train_y)

    text_list = ['我是中国人',
                 '我爱北京天安门',
                 '郭小美和王帅身穿和服走在大街上',
                 '李冰冰从马上跳下来',
                 '武汉市长江大桥发表重要讲话',
                 '人们常说生活是一部教科书，而血与火的战争更是不可多得的教科书，她确实是名副其实的‘我的大学’。',
                 '为了有效地解决“高产穷县”的矛盾，吉林省委、省政府深入实际，调查研究，确定了实施“三大一强”的农业发展战略，即经过的努力，'
                 '粮食产量要再上两个台阶，畜牧业要成为农民增收的支柱产业，农副产品加工业要成为全省工业和财政收入的一大支柱，真正成为粮食"'
                 ]

    for text in text_list:
        print(crf_seg.seg(text))

Epoch 1/10
153/153 [==============================] - 176s 1s/step - loss: 29.2612 - get_accuracy: 0.2364 - val_loss: 27.3445 - val_get_accuracy: 0.2635
Epoch 2/10
153/153 [==============================] - 170s 1s/step - loss: 23.7151 - get_accuracy: 0.4246 - val_loss: 19.2997 - val_get_accuracy: 0.7477
Epoch 3/10
153/153 [==============================] - 171s 1s/step - loss: 14.5272 - get_accuracy: 0.8200 - val_loss: 11.0084 - val_get_accuracy: 0.8842
Epoch 4/10
153/153 [==============================] - 169s 1s/step - loss: 8.9450 - get_accuracy: 0.9023 - val_loss: 8.0100 - val_get_accuracy: 0.9019
Epoch 5/10
153/153 [==============================] - 170s 1s/step - loss: 6.6793 - get_accuracy: 0.9183 - val_loss: 6.3761 - val_get_accuracy: 0.9135
Epoch 6/10
153/153 [==============================] - 170s 1s/step - loss: 5.3494 - get_accuracy: 0.9289 - val_loss: 5.3250 - val_get_accuracy: 0.9218
Epoch 7/10
153/153 [==============================] - 170s 1s/step - loss: 4.4426 - get_

In [56]:
import operator
def evaluation(prophet, gold_path):
    data_list = []
    gold_list = []

    reader = open(gold_path, 'r')
    while True:
        line = reader.readline()
        if not line:
            break
        line = line.rstrip()
        data_list.append(line.replace('  ', ''))
        gold_list.append(line)

    reader.close()

    correct = 0
    size = len(data_list)
    a = 0
    b = 0
    a_and_b = 0
    for i in range(0, size):
        result = prophet(data_list[i])
        if result is None:
            continue
        b += len(result)

        gold_str = gold_list[i]
        gold = gold_str.split("  ")
        a += len(gold)

        corr = operator.eq(gold, result)
        if corr:
            correct += 1
        else:
            print(i)
            print(' '.join(result))
            print(' '.join(gold))

        l = list(set(result).intersection(set(gold)))
        a_and_b += len(l)

    print(" correct : %f  %f  P: %f  R: %f" % (correct, correct * 1.0 / size ,  a_and_b * 1.0 / b ,  a_and_b * 1.0 / a))

def evaluation_seg(segment, gold_path):
    def prophet(text):
        return segment.seg(text)

    evaluation(prophet, gold_path)

In [ ]:
#from nlp.segment.tools import evaluation_seg

seg = build_segment()
gold = data_path+"/nlp/corpus/icwb2-data/gold/msr_test_gold.utf8"
evaluation_seg(seg, gold)

In [61]:
!git add .
!git commit -m ''